functions - not oneliners
math - not just functions

In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, MinMaxScaler
warnings.filterwarnings('ignore')
data = pd.read_csv('''C:\\Users\\user\\Documents\\GitHub\\notebooks-on-ml\\Coursera lil proj\\DATA\\a_steam_data_2021_2025.csv''')

In [3]:
data['genres'] = data['genres'].fillna(data.groupby('categories')['genres'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan))
data['genres'] = data['genres'].replace(['nan', 'None', ''], np.nan)
data['categories'] = data['categories'].replace(['nan', 'None', ''], np.nan)
data = data.dropna(subset=['genres', 'categories'])
data['developer'] = data['developer'].fillna('Noname')
data['publisher'] = data['publisher'].fillna('Noname')

In [4]:
data['release_date'] = pd.to_datetime(data['release_date'], format='mixed', errors='coerce')
data['release_date'] = data['release_date'].fillna(pd.to_datetime('Aug 1 ' + data['release_year'].astype(str), format='%b %d %Y', errors='coerce'))       
data['genres'] = data['genres'].astype(str).str.split(';')
data['categories'] = data['categories'].astype(str).str.split(';')
data = data.sort_values(['release_date']).reset_index(drop=True)

In [12]:
data['is_sequel'] = data['name'].apply(lambda x: 1 if any(word.upper() in ['2', '3', 'II', 'III', 'IV', 'V'] for word in str(x).split()) else 0).astype(int)
data['developer&publisher'] = [1 if any(i in p for i in d) else 0 for d, p in zip(data['developer'], data['publisher'])]

data['genre_count'] = data['genres'].apply(len)
data['categories_count'] = data['categories'].apply(len)

data['niche'] = data['genres'].str[0] + '_' + data['categories'].str[0]
dev_gen_seen, dev_cat_seen = {}, {}
dg_hist, dc_hist = [], []

dev_gen_seen, dev_cat_seen = {}, {}
pub_gen_seen, pub_cat_seen = {}, {}
dg_hist, dc_hist, pg_hist, pc_hist = [], [], [], []
for d, p, g, c in zip(data['developer'], data['publisher'], data['genres'], data['categories']):
    d_key, p_key = tuple(d) if isinstance(d, list) else d, tuple(p) if isinstance(p, list) else p
    dg_hist.append(list(dev_gen_seen.get(d_key, [])))
    dc_hist.append(list(dev_cat_seen.get(d_key, [])))
    pg_hist.append(list(pub_gen_seen.get(p_key, [])))
    pc_hist.append(list(pub_cat_seen.get(p_key, [])))
    dev_gen_seen.setdefault(d_key, set()).update(g)
    dev_cat_seen.setdefault(d_key, set()).update(c)
    pub_gen_seen.setdefault(p_key, set()).update(g)
    pub_cat_seen.setdefault(p_key, set()).update(c)

data['developer_genres'] = dg_hist
data['developer_categories'] = dc_hist
data['publisher_genres'] = pg_hist
data['publisher_categories'] = pc_hist
data['unique_genres'] = [len(x) for x in dg_hist]
data['unique_categories'] = [len(x) for x in dc_hist]

In [13]:
data['log_price'] = np.log1p(data['price'])
data['log_recommendations'] = np.log1p(data['recommendations'])

In [ ]:
data['is_hit'] = data['recommendations'] > 800
#data['is_mega_hit'] = data['recommendations'] > 1800
data['is_discount'] = (data['price'] % 1) > 0.90
data['developer_hit_ratio'] = data.groupby('developer')['is_hit'].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
data['publisher_hit_ratio'] = data.groupby('publisher')['is_hit'].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
data['developer_game_number'] = data.groupby('developer').cumcount() + 1
data['publisher_game_number'] = data.groupby('publisher').cumcount() + 1
#data['timeliness'] = data['recommendations'] / ((2026 - data['release_date'].astype(str).str.split('-').str[0].astype(int)) * 12 - (data['release_date'].astype(str).str.split('-').str[1].astype(int) - 1)).replace(0, 1)
data['release_month'] = data['release_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['release_month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['release_month'] / 12)
data['is_new_niche'] = ~data.duplicated(['developer', 'niche'])
data['total_niches_count'] = data.groupby('developer')['is_new_niche'].transform(lambda x: x.shift(1).cumsum()).fillna(0)
data['is_specialist'] = (data['total_niches_count'] <= 2).astype(int)
data['is_veteran'] = (data['niche_count'] >= 3).astype(int)

data['developer_genre_overlap'] = [len(set(g) & set(h)) / len(set(g)) if len(g) > 0 else 0 for g, h in zip(data['genres'], data['developer_genres'])]
data['developer_category_overlap'] = [len(set(c) & set(h)) / len(set(c)) if len(c) > 0 else 0 for c, h in zip(data['categories'], data['developer_categories'])]
data['publisher_genre_overlap'] = [len(set(g) & set(h)) / len(set(g)) if len(g) > 0 else 0 for g, h in zip(data['genres'], data['publisher_genres'])]
data['publisher_category_overlap'] = [len(set(c) & set(h)) / len(set(c)) if len(c) > 0 else 0 for c, h in zip(data['categories'], data['publisher_categories'])]
data['publisher_niche_count'] = data.groupby(['publisher', 'niche']).cumcount()
data['publisher_niche_count'] = data.groupby(['publisher', 'niche'])['publisher_niche_count'].shift(1).fillna(0)
data['developer_niche_count'] = data.groupby(['developer', 'niche']).cumcount()
data['developer_niche_count'] = data.groupby(['developer', 'niche'])['developer_niche_count'].shift(1).fillna(0)
data['developer_std'] = data.groupby('developer')['price'].transform(lambda x: x.shift(1).expanding().std()).fillna(0)
data['publisher_std'] = data.groupby('publisher')['price'].transform(lambda x: x.shift(1).expanding().std()).fillna(0)
data['price_per_genre'] = data['log_price'] / (data['genre_count'] + 1)

data['developer_price'] = data.groupby('developer')['price'].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
data['publisher_price'] = data.groupby('publisher')['price'].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
data['developer_recommendations'] = data.groupby('developer')['recommendations'].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
data['publisher_recommendations'] = data.groupby('publisher')['recommendations'].transform(lambda x: x.shift(1).expanding().mean()).fillna(0)
#data['quality'] = data['recommendations'] / (data['price'] + 0.001)
#data['developer_quality_ratio'] = data.groupby('developer')['quality'].transform(lambda x: x.shift(1).expanding().mean())
#data['publisher_quality_ratio'] = data.groupby('publisher')['quality'].transform(lambda x: x.shift(1).expanding().mean())
data['developer_momentum'] = data.groupby('developer')['recommendations'].transform(lambda x: x.diff().shift(1)).fillna(0)
#data['developer_niche_recommendations'] = data.groupby(['developer', 'niche'])['recommendations'].transform(lambda x: x.shift(1).expanding().mean())
data['publisher_momentum'] = data.groupby('publisher')['recommendations'].transform(lambda x: x.diff().shift(1)).fillna(0)
#data['publisher_niche_recommendations'] = data.groupby(['publisher', 'niche'])['recommendations'].transform(lambda x: x.shift(1).expanding().mean())

In [8]:
data['log_developer_price'] = np.log1p(data['developer_price'])
#data['log_publisher_price'] = np.log1p(data['publisher_price'])
data['log_developer_recommendations'] = np.log1p(data['developer_recommendations'])
data['log_publisher_recommendations'] = np.log1p(data['publisher_recommendations'])
#data['log_quality'] = np.log1p(data['quality'])
#data['log_developer_quality_ratio'] = np.log1p(data['developer_quality_ratio'])
#data['log_publisher_quality_ratio'] = np.log1p(data['publisher_quality_ratio'])
data['log_developer_momentum'] = np.sign(data['developer_momentum']) * np.log1p(np.abs(data['developer_momentum']))
#data['log_developer_niche_recommendations'] = np.log1p(data['developer_niche_recommendations'])
data['log_publisher_momentum'] = np.sign(data['publisher_momentum']) * np.log1p(np.abs(data['publisher_momentum']))
#data['log_publisher_niche_recommendations'] = np.log1p(data['publisher_niche_recommendations'])

data['developer_count'] = data['developer'].map(data['developer'].value_counts())
data['log_developer_count'] = np.log1p(data['developer_count'])
data['publisher_count'] = data['publisher'].map(data['publisher'].value_counts())
data['log_publisher_count'] = np.log1p(data['publisher_count'])

In [12]:
#inf_counts = np.isinf(data.select_dtypes(include=[np.number])).sum()
#print(inf_counts[inf_counts > 0])
#print('----------------')
#nan_counts = data.isnull().sum()
#print(nan_counts[nan_counts > 0])

In [10]:
robust_cols = [
    'log_price', 'log_developer_price',
    'log_developer_recommendations', 'log_publisher_recommendations', 
#    'log_quality', 'log_developer_quality_ratio', 'log_publisher_quality_ratio', 
#    'log_publisher_price', 'log_recommendations',
    'log_developer_momentum', 'log_publisher_momentum',
#    'log_developer_niche_recommendations', 'log_publisher_niche_recommendations',
    'developer_niche_count', 'publisher_niche_count',
    'log_developer_count', 'log_publisher_count'
]
minmax_cols = [
#    'timeliness', 
    'genre_count', 'categories_count', 
    'developer_std', 'publisher_std', 
    'unique_genres', 'unique_categories',
    'developer_genre_overlap', 'developer_category_overlap',
    'publisher_genre_overlap', 'publisher_category_overlap'
]
robust = RobustScaler()
data[robust_cols] = robust.fit_transform(data[robust_cols])
mms = MinMaxScaler()
data[minmax_cols] = mms.fit_transform(data[minmax_cols])

In [11]:
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(data['genres'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=data.index)
category_encoded = mlb.fit_transform(data['categories'])
category_df = pd.DataFrame(category_encoded, columns=mlb.classes_, index=data.index)
data = pd.concat([data, genre_df, category_df], axis=1)
data['mobile'] = ((data['Remote Play on Phone'] == 1) | (data['Remote Play on Tablet'] == 1)).astype(int)

In [12]:
data = data.drop([
    'publisher', 'developer', 'publisher_count', 'developer_count', 'Remote Play on Tablet', 
    'price', 'recommendations', 'genres', 'categories', 'name', 'niche', 'Remote Play on Phone',
    'publisher_genres', 'publisher_categories', 'developer_genres',  'appid',
    'developer_momentum', 'publisher_momentum', 'developer_categories', 'release_date',
    'publisher_price', 'developer_price', 'developer_recommendations', 'log_developer_recommendations',
    'publisher_recommendations', 'log_publisher_recommendations', 'release_month',
    'Family Sharing', 'PvP', 'Co-op', 'Shared/Split Screen', 'log_recommendations'
], axis=1)

In [ ]:
data.to_csv('transition_file.csv', index=False)

In [ ]:
data = pd.read_csv('transition_file.csv')

In [13]:
X = data.drop('is_hit', axis=1)
y = data['is_hit']

In [15]:
corr_matrix = data.select_dtypes(include=[np.number]).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(f"Redundant columns to drop: {to_drop}")

Redundant columns to drop: ['developer_niche_count']


In [16]:
pairs = upper.unstack().dropna()
sorted_pairs = pairs.sort_values(ascending=False)
print("Top 10 most redundant pairs:")
print(sorted_pairs.head(10))

Top 10 most redundant pairs:
developer_niche_count  niche_count                   1.000000
developer_game_number  niche_count                   0.875792
developer_niche_count  developer_game_number         0.875792
publisher_niche_count  publisher_game_number         0.853333
price_per_genre        log_price                     0.833792
log_publisher_count    log_developer_count           0.774113
VR Only                Tracked Controller Support    0.751727
unique_categories      categories_count              0.751136
log_developer_count    total_niches_count            0.748377
publisher_niche_count  niche_count                   0.732975
dtype: float64


sns.scatterplot(x=data['publisher_std'], y=data['developer_std'])
plt.title("Visualizing Redundancy")
plt.show()

plt.figure(figsize=(15, 10))
subset = data.iloc[:, :30] 
sns.heatmap(subset.corr(), annot=False, cmap='coolwarm')
plt.show()

plot_data = data.explode('genres').explode('categories')
sns.barplot(data=plot_data, x='release_year', y='price')
plt.show()
sns.barplot(data=plot_data, x='release_year', y='recommendations')
plt.show()

timeline_data = plot_data.groupby(['release_year', 'genres']).size().reset_index(name='count')
timeline_data_2 = plot_data.groupby(['release_year', 'categories']).size().reset_index(name='count')
sns.lineplot(data=timeline_data, x='release_year', y='count', hue='genres')
plt.show()
sns.lineplot(data=timeline_data_2, x='release_year', y='count', hue='categories')
plt.show()

sns.barplot(data=plot_data, x='price', y='genres')
plt.show()
sns.barplot(data=plot_data, x='price', y='categories')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='genres')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='categories')
plt.show()

sns.histplot(data['price'], kde=True)
plt.show()
sns.histplot(data['recommendations'], kde=True)
plt.show()
sns.histplot(data['log_price'], kde=True)
plt.show()
sns.histplot(data['log_recommendations'], kde=True)
plt.show()

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import optuna

base_models = [
    ('xgb', XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=5)),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=10, min_sample_leaf=5))
]
up_model = LogisticRegression(c=0.1, max_iter=1000)
stack = StackingClassifier(
    estimators=base_models,
    deciders=up_model,
    cv=5,
    stack_method='predict_proba',
    passthrough=False
)

multi_model = MultiOutputClassifier(stack)
y = data[['is_hit', 'is_sequel']]
multi_model.fit(X, y)